# Import Sentence-Transformer using PhoBERT


In [1]:
%pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!gdown https://drive.google.com/a/gm.uit.edu.vn/uc?id=1pXJZ9eHp6DWkQ5MhCzmWYsKyLQEDiodz&export=download
!tar xzf /content/vn_sbert_deploy.tar.gz

Downloading...
From: https://drive.google.com/a/gm.uit.edu.vn/uc?id=1pXJZ9eHp6DWkQ5MhCzmWYsKyLQEDiodz
To: /content/vn_sbert_deploy.tar.gz
100% 485M/485M [00:08<00:00, 60.4MB/s]


In [3]:
!pip -q install transformers
!pip -q install vncorenlp
!pip -q install fairseq
!pip -q install fastBPE

!pip -q install fastapi
!pip -q install uvicorn
!pip -q install pyngrok
!pip -q install pyyaml==5.4.1 

     |████████████████████████████████| 4.2 MB 37.2 MB/s 
     |████████████████████████████████| 596 kB 59.0 MB/s 
     |████████████████████████████████| 6.6 MB 51.2 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 2.6 MB 38.0 MB/s 
     |████████████████████████████████| 1.7 MB 14.3 MB/s 
     |████████████████████████████████| 151 kB 69.0 MB/s 
     |████████████████████████████████| 92 kB 12.4 MB/s 
     |████████████████████████████████| 117 kB 74.0 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 11.1 MB 52.7 MB/s 
     |████████████████████████████████| 80 kB 10.5 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 58 kB 7.1 MB/s 
     |████████████████████████████████| 745 kB 19.3 MB/s 
     |█████████████████████

In [4]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget -q --show-progress https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.1s    
vi-vocab            100%[===================>] 514.20K  --.-KB/s    in 0.004s  
wordsegmenter.rdr   100%[===================>] 125.50K  --.-KB/s    in 0.002s  


In [5]:
!git clone https://github.com/DatCanCode/sentence-transformers
!cd sentence-transformers; pip install -e .

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 1200, done.
remote: Total 1200 (delta 0), reused 0 (delta 0), pack-reused 1200
Receiving objects: 100% (1200/1200), 30.67 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (820/820), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/sentence-transformers
  Running setup.py develop for sentence-transformers


In [6]:
%cd /content/sentence-transformers/
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('/content/vn_sbert_deploy/phobert_base_mean_tokens_NLI_STS')

/content/sentence-transformers


In [7]:
# !pip install sentence-transformers
# !pip install transformers
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 18.6 MB/s 
     |████████████████████████████████| 965 kB 54.5 MB/s 
     |████████████████████████████████| 235 kB 5.5 MB/s 
     |████████████████████████████████| 581 kB 22.0 MB/s 


# Helper functions

In [8]:
query = ['Cô giáo đang ăn kem.']
query_embedding = np.array(model.encode(query))
corpus = ['Cô giáo đang ăn kem.',
          'Cô giáo đang ăn bánh mì.',
          'Chị gái đang thử món thịt dê.',
          'Một anh trai đang cưỡi ngựa.',
          'Kị binh đang đi tuần.',
          'Một vụ tai nạn thảm khốc vừa xảy ra.',
          'Đã có ít nhất hai người chết trong vụ xe khách rơi xuống vực.',
          'Một con khỉ đang diễn xiếc.',
          'Một người đàn ông trong bộ đồ tinh tinh đang làm trò.',
          'Lũ quét đột ngột gây thiệt hại cho ít nhất 3 tỉnh miền núi']

corpus_embeddings = np.array(model.encode(corpus))

In [9]:
import torch
def cos_sim(corpus, queries):
  # corpus: N x d 
  # queries: K x d
  # score matrix: N x K
  norm_corpus = np.linalg.norm(corpus, axis=1, keepdims=True)
  norm_query = np.linalg.norm(queries, axis=1, keepdims=True)
  scores = np.dot(corpus/norm_corpus, (queries/norm_query).T)
  return scores 

In [10]:
cos_sim(corpus_embeddings, query_embedding)[0]

array([1.], dtype=float32)

In [11]:
import sklearn
sklearn.metrics.pairwise.cosine_similarity(query_embedding, corpus_embeddings)

array([[ 0.99999976,  0.6672883 ,  0.35625407,  0.08035747,  0.10856581,
         0.12286636,  0.13099083,  0.13471115,  0.0465546 , -0.06717575]],
      dtype=float32)

In [12]:
cxt = "Ngay từ đầu những năm 1940, phần cứng và phần mềm đã được sử dụng trong giáo dục và đào tạo, khi đó các nhà nghiên cứu Mỹ phát triển các mô hình tập bay sử dụng máy tính analog để tạo ra các mô phỏng cài đặt trong thiết bị dữ liệu. Một hệ thống thuộc loại này là the type19 synthetic radar trainer được xây dựng vào năm 1943. Từ những cố gắng ban đầu này, trong giai đoạn từ chiến tranh thế giới II đến giữa 1970, phần mềm giáo dục được cài đặt trực tiếp vào phần cứng, thường là các máy tính lớn. Người đi tiên phong trong giai đoạn này là PLATO (1960) được phát triển ở đại học Illinois và TICCIT (1969). Các thiết bị này có giá trên 10.000, chúng vượt ra ngoài khả năng tài chính của hầu hết các cơ quan. Một số ngôn ngữ lập trình trong giai đoạn này, đặc biệt BASIC (1963) và LOGO (1967), có thể dạy cho sinh viên và những người mới làm quen với máy tính. Plato IV, được phát hành vào 1972, hỗ trợ rất nhiều tính năng mà sau này trở thành chuẩn cho phần mềm giáo dục chạy ở máy tính trong gia đình. Những tính năng này gồm có các ảnh đồ họa, tạo ra âm thanh và hỗ trợ các thiết bị vào không qua bàn phím (như là chạm vào màn hình)."
            

In [13]:
from underthesea import sent_tokenize
import json
cxt_sen = sent_tokenize(cxt)
cxt_sen

['Ngay từ đầu những năm 1940, phần cứng và phần mềm đã được sử dụng trong giáo dục và đào tạo, khi đó các nhà nghiên cứu Mỹ phát triển các mô hình tập bay sử dụng máy tính analog để tạo ra các mô phỏng cài đặt trong thiết bị dữ liệu.',
 'Một hệ thống thuộc loại này là the type19 synthetic radar trainer được xây dựng vào năm 1943.',
 'Từ những cố gắng ban đầu này, trong giai đoạn từ chiến tranh thế giới II đến giữa 1970, phần mềm giáo dục được cài đặt trực tiếp vào phần cứng, thường là các máy tính lớn.',
 'Người đi tiên phong trong giai đoạn này là PLATO (1960) được phát triển ở đại học Illinois và TICCIT (1969).',
 'Các thiết bị này có giá trên 10.000, chúng vượt ra ngoài khả năng tài chính của hầu hết các cơ quan.',
 'Một số ngôn ngữ lập trình trong giai đoạn này, đặc biệt BASIC (1963) và LOGO (1967), có thể dạy cho sinh viên và những người mới làm quen với máy tính.',
 'Plato IV, được phát hành vào 1972, hỗ trợ rất nhiều tính năng mà sau này trở thành chuẩn cho phần mềm giáo dục chạ

In [14]:
def top_k(top_k, model, question, corpus, corpus_embeddings):
      k = min(len(corpus), top_k)
      query_embedding = np.array(model.encode([question]))
      cos_scores = sklearn.metrics.pairwise.cosine_similarity(query_embedding, corpus_embeddings)
      cos_scores = torch.Tensor(cos_scores[0])
      top_results = torch.topk(cos_scores, k=k)

      return top_results[1]

# Update train data

In [15]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [16]:
%cd "/content/drive/MyDrive/vdt_dsai/dataset"

/content/drive/MyDrive/vdt_dsai/dataset


In [17]:
!ls

test_file.json	train_file.json  val_file.json


In [18]:
with open("test_file.json") as f:
  test_data =  json.load(f)

In [19]:
from copy import deepcopy
def update_top_10(train_data, k):
  # iter = 30
  copied_data = deepcopy(train_data)
  for topic in copied_data['data']:
    for paragraph in topic['paragraphs']:
      # paragraph['context'] = sent_tokenize(paragraph['context'])
      # print(paragraph['context'])
      context = paragraph['context']
      context_embeddings = model.encode(context)
      for qa in paragraph['qas']:
        question = qa['question']

        ## Get top k most relevant sentences
        top_k_candidates = top_k(k, model, question, context, context_embeddings)
        qa['candidate_indices'] = top_k_candidates.tolist()
    # iter = iter - 1
    # if (iter == 0):
    #   return copied_data
  return copied_data
  
      

In [20]:
# new_data_train = update_data_train(5, data)
top_10_data = update_top_10(test_data, 10)
# json_object = json.dumps(top_10_data, ensure_ascii=False)
# with open("phobert_top_10_train.json", "w") as write_file:
#   write_file.write(json_object)
top_5_data = update_top_10(test_data, 5)

# Calculate accuracy


In [21]:
!ls

test_file.json	train_file.json  val_file.json


In [22]:
def find_sentence(corpus, start_answer):
    '''
    Find sentence contains answer
    * corpus: list sentences of context
    * start_answer: the begin position of answer in context
    '''
    start, end = 0, -1
    sent_idx = -1
    for i in range(len(corpus)):
        sentence = corpus[i]
        start = end + 1
        end = start+len(sentence)
        if start<=start_answer<end:
            sent_idx = i
            break
        elif start_answer == end:
            sent_idx = i + 1
            break 
    return sent_idx

def accuracy(data, top_k):
    """
    Calculate accuracy 
    Given queries Q = {Q1, ..., Qm} 
    Document D = {S1, ..., Sn}
    The sentence containing the answer to the question Qi is Ai
    
    Acc@K = 1/|Q| * sum(is Ai in get_top_k(Q, D))
    """
    results = []
    invalid = 0 # number of invalid qa
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0:

                    # If the answer is from multiple sentences then discard this qa
                    if len(sent_tokenize(qa['answers'][0]['text'])) > 1:
                        invalid += 1
                        continue 

                    start_answer = qa['answers'][0]['answer_start']
                    k = min(top_k, len(qa['candidate_indices']))

                    # Find the index of sentence that contains the answer 
                    ans_sent_idx = find_sentence(context, start_answer)
                    # Check if the above sentence is retrived in top_k 
                    results.append(ans_sent_idx in qa['candidate_indices'][:k])
    return {"top_k":top_k, "# valid qa":len(results), "# invalid qa": invalid,  "true":sum(results), "accuracy":round(sum(results) / len(results), 3)}

In [24]:
print("PhoBERT Test")
phobert_test_results = []
for i in range(10):
    json_i = accuracy(top_10_data, i+1)
    phobert_test_results.append(json_i)
import pandas as pd 
phobert_test_df = pd.read_json(json.dumps(phobert_test_results))
phobert_test_df

PhoBERT Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1113,0.654
1,2,1701,3,1411,0.830
2,3,1701,3,1552,0.912
3,4,1701,3,1624,0.955
4,5,1701,3,1675,0.985
5,6,1701,3,1688,0.992
6,7,1701,3,1695,0.996
7,8,1701,3,1699,0.999
8,9,1701,3,1700,0.999
9,10,1701,3,1701,1.000
